In [1]:
import numpy as np 
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# import os
# import re
# import math
# import plotly.express as px
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots

# from lightgbm import LGBMRegressor, LGBMClassifier
# from xgboost import XGBRegressor, XGBClassifier
# from catboost import CatBoostRegressor, CatBoostClassifier
# import lightgbm as lgb
# import xgboost as xgb
# import catboost as cab

# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold, GridSearchCV
# from sklearn.decomposition import LatentDirichletAllocation, NMF, TruncatedSVD
# from sklearn.linear_model import LogisticRegression, SGDClassifier
# from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor, StackingRegressor
# from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, StackingClassifier
# from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
# from sklearn import metrics
# from sklearn.svm import SVC
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
# from collections import defaultdict, Counter
import warnings
import json 
import pickle
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import pickle
import random
from tqdm import tqdm
# import sentence_transformers 
# from sklearn.preprocessing import KBinsDiscretizer
# from sentence_transformers import SentenceTransformer

# from gensim.models import Word2Vec

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import random_split, DataLoader
from torch.utils.data import Dataset
import argparse
from utils import set_random_seed, get_logger, ensure_dir, str2bool, str2float
from data import NNDataset, NNDatasetV2
from model import MatchModel, BaseModel, MatchModelV2

"""
相比于trainDeeep.py，加入一些手动聚合的序列特征，例如历史序列的平均价格，历史序列的不同类别数之类的
SeqFeatureEmbedding现在使用2层全连接，可以换多层！
# TODO: product-id作为特征之一
# TODO: emb_dim变大 不同特征用不同emb_dim等
# TODO: DIN等序列模型
"""


emb_dim = 16
dense_bins = 10
hid_dim = 256
dropout = 0.1
layers = 4
bidirectional = False
seq_emb_factor = 4  # 人工序列特征的嵌入是emb_dim的几倍

batch_size = 1024
epochs = 100
len_candidate_set = 100
device = torch.device('cuda:3')
dense_norm = True
num_workers = 0

load_init = True

load_exp_id = [
    93545,
    43234,
    84653,
    29020,
]
load_epoch = [
    52,
    51,
    53,
    50
]
load_Fold = [
    0,
    1,
    2,
    3
]

# TODO: 调整batch-size， 调整hidden-size
# TODO: 跑Fold 4
learning_rate = 0.001
weight_decay = 0.00001
early_stop_lr = 1e-6
lr_patience = 5
lr_decay_ratio = 0.1
clip = 5
log_every = 100
early_stop = True
patience = 10
kfold = 5
attn_match = True 

w2v_window = 3
w2v_min_count = 1
w2v_epochs = 500
w2v_vector_size = 128

seed = 2023
set_random_seed(seed)

model_name = 'MatchModelV2withATTMatch'
loc2id = {'DE': 0, 'JP': 1, 'UK': 2, 'ES': 3, 'FR': 4, 'IT': 5}

config = locals()

# 加载必要的数据

exp_id = config.get('exp_id', None)
if exp_id is None:
    exp_id = int(random.SystemRandom().random() * 100000)
    config['exp_id'] = exp_id

logger = get_logger(config)
logger.info('Exp_id {}'.format(exp_id))
logger.info(config)

logger.info('load_exp_id = {}'.format(load_exp_id))
logger.info('load_epoch = {}'.format(load_epoch))

logger.info('read data')

titles_embedding = np.load('./data/titles_embedding.npy')
descs_embedding = np.load('./data/descs_embedding.npy')
logger.info('titles_embedding: {}'.format(titles_embedding.shape))
logger.info('descs_embedding: {}'.format(descs_embedding.shape))

product2id = json.load(open('data/product2id.json', 'r'))
id2product = json.load(open('data/id2product.json', 'r'))
id2product = {int(k): v for k, v in id2product.items()}
logger.info('product2id: {}'.format(len(product2id)))
logger.info('id2product: {}'.format(len(id2product)))

word2vec_embedding = np.load('./data/word2vec_embedding.npy')
logger.info('word2vec_embedding: {}'.format(word2vec_embedding.shape))

top200 = pickle.load(open('data/top200_new.pkl', 'rb'))

df_train_encoded = pd.read_csv('data/df_train_encoded.csv')
df_test_encoded = pd.read_csv('data/df_test_encoded.csv')
products_encoded = pd.read_csv('./data/products_encoded.csv')
logger.info('df_train_encoded: {}'.format(df_train_encoded.shape))
logger.info('df_test_encoded: {}'.format(df_test_encoded.shape))
logger.info('products_encoded: {}'.format(products_encoded.shape))

num_features = ['price', 'len_title', 'len_desc']
if dense_norm:
    logger.info('MinMaxScaler Norm products_num_feas')
    mms = MinMaxScaler(feature_range=(0,1))
    products_encoded[num_features] = mms.fit_transform(products_encoded[num_features])
for fe in num_features:
    products_encoded.loc[:, fe] = products_encoded.loc[:, fe].astype('float32')

logger.info('Load Hand-made Seq Features')
df_train_seqs_feas_all = pd.read_csv('data/df_train_seqs_feas_all.csv')  # 29维特征
df_test_seqs_feas_all = pd.read_csv('data/df_test_seqs_feas_all.csv')
logger.info('df_train_seqs_feas_all: {}'.format(df_train_seqs_feas_all.shape))
logger.info('df_test_seqs_feas_all: {}'.format(df_test_seqs_feas_all.shape))
seqs_cat_feas = [f for f in df_train_seqs_feas_all.columns if 'NUNIQUE' in f or 'COUNT' in f]
seqs_num_feas = [f for f in df_train_seqs_feas_all.columns if f not in seqs_cat_feas]
logger.info('seqs_cat_feas: {}'.format(seqs_cat_feas))
logger.info('seqs_num_feas: {}'.format(seqs_num_feas))

if dense_norm:
    logger.info('MinMaxScaler Norm seqs_num_feas')
    mms = MinMaxScaler(feature_range=(0,1))
    df_train_seqs_feas_all[seqs_num_feas] = mms.fit_transform(df_train_seqs_feas_all[seqs_num_feas])
    df_test_seqs_feas_all[seqs_num_feas] = mms.fit_transform(df_test_seqs_feas_all[seqs_num_feas])
for fe in seqs_num_feas:
    df_train_seqs_feas_all.loc[:, fe] = df_train_seqs_feas_all.loc[:, fe].astype('float32')
    df_test_seqs_feas_all.loc[:, fe] = df_test_seqs_feas_all.loc[:, fe].astype('float32')

df_train_seqs_cat_feas = df_train_seqs_feas_all[seqs_cat_feas]
df_train_seqs_num_feas = df_train_seqs_feas_all[seqs_num_feas]
df_test_seqs_cat_feas = df_test_seqs_feas_all[seqs_cat_feas]
df_test_seqs_num_feas = df_test_seqs_feas_all[seqs_num_feas]
logger.info('df_train_seqs_cat_feas: {}'.format(df_train_seqs_cat_feas.shape))
logger.info('df_train_seqs_num_feas: {}'.format(df_train_seqs_num_feas.shape))
logger.info('df_test_seqs_cat_feas: {}'.format(df_test_seqs_cat_feas.shape))
logger.info('df_test_seqs_num_feas: {}'.format(df_test_seqs_num_feas.shape))

id_count = products_encoded.shape[0]

train_preds_encoded = pickle.load(open('./data/train_preds_all_encoded_new.pkl', 'rb'))  # (len_train, 100)
test_preds_encoded = pickle.load(open('./data/test_preds_all_encoded_new.pkl', 'rb'))  # (len_test, 100)
test_preds = pickle.load(open('./data/test_preds_all.pkl', 'rb'))
logger.info('train_preds_encoded: {}'.format(len(train_preds_encoded)))
logger.info('test_preds_encoded: {}'.format(len(test_preds_encoded)))
logger.info('test_preds: {}'.format(len(test_preds)))

logger.info('Cutting the candidate_set to {}'.format(len_candidate_set))
cut_train_preds_encoded = [lst[:len_candidate_set] for lst in tqdm(train_preds_encoded, total=len(train_preds_encoded))]
df_train_encoded['recall'] = cut_train_preds_encoded
cut_test_preds_encoded = [lst[:len_candidate_set] for lst in tqdm(test_preds_encoded, total=len(test_preds_encoded))]
df_test_encoded['recall'] = cut_test_preds_encoded

logger.info('Eval the prev_items')
df_train_encoded['prev_items'] = df_train_encoded['prev_items'].apply(eval)
df_test_encoded['prev_items'] = df_test_encoded['prev_items'].apply(eval)

df_test = pd.read_csv('data/sessions_test_task1.csv')
logger.info('df_test: {}'.format(df_test.shape))

tmp = pd.concat([df_train_seqs_feas_all[seqs_cat_feas], df_test_seqs_feas_all[seqs_cat_feas]])
tmp_nunique = (tmp.max() + 1).to_dict()  # 不是nunique，因为这个是计数特征，不是连续的0~n-1

data_feature = {}
data_feature['len_encode_brand'] = products_encoded['encode_brand'].nunique()
data_feature['len_encode_color'] = products_encoded['encode_color'].nunique()
data_feature['len_encode_size'] = products_encoded['encode_size'].nunique()
data_feature['len_encode_model'] = products_encoded['encode_model'].nunique()
data_feature['len_encode_material'] = products_encoded['encode_material'].nunique()
data_feature['len_encode_author'] = products_encoded['encode_author'].nunique()
data_feature['len_locale'] = len(loc2id)
data_feature['dense_bins'] = dense_bins
data_feature['id_count'] = id_count
data_feature['len_features'] = products_encoded.shape[1] - 1
data_feature['len_emb_features'] = 3
data_feature['len_candidate_set'] = len_candidate_set
data_feature['w2v_vector_size'] = w2v_vector_size
data_feature['sentence_vector_size'] = 384
data_feature['len_seqs_cat_feas'] = len(seqs_cat_feas)
data_feature['len_seqs_num_feas'] = len(seqs_num_feas)
data_feature['seq_emb_factor'] = seq_emb_factor
data_feature.update(tmp_nunique)
logger.info('data_feature:')
logger.info(data_feature)

del tmp

# 加载模型等

logger.info('create model')

products_input = {name: torch.tensor(products_encoded[name].values).to(device) for name in products_encoded.columns}

if 'BaseModel' in model_name:
    model = BaseModel(config, data_feature, products_input, word2vec_embedding, titles_embedding, descs_embedding).to(device)
elif 'MatchModel' in model_name:
    model = MatchModelV2(config, data_feature, products_input, word2vec_embedding, titles_embedding, descs_embedding).to(device)
else:
    raise ValueError('Error model name {}'.format(model_name))
logger.info(model)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='max', patience=lr_patience, factor=lr_decay_ratio)

for name, param in model.named_parameters():
    logger.info(str(name) + '\t' + str(param.shape) + '\t' +
                              str(param.device) + '\t' + str(param.requires_grad))
total_num = sum([param.nelement() for param in model.parameters()])
logger.info('Total parameter numbers: {}'.format(total_num))


# 数据集DataLoader
train_set = NNDatasetV2(df_train_encoded, df_train_seqs_cat_feas, 
                       df_train_seqs_num_feas)
logger.info('train_set: {}'.format(len(train_set)))
test_set = NNDatasetV2(df_test_encoded, df_test_seqs_cat_feas, 
                       df_test_seqs_num_feas)
logger.info('test_set: {}'.format(len(test_set)))


def collate_fn(indices):
    batch_prev_items = []
    batch_locale = []
    batch_candidate_set = []
    batch_len = []
    batch_mask = []
    batch_label = []
    batch_label_index = []  # 交叉熵需要的是label在候选集中的index
    batch_seq_cat = []
    batch_seq_num = []
    for item in indices:
        batch_len.append(len(item[0]))  # prev_items
    max_len = max(batch_len)
    for item in indices:
        l = len(item[0])
        batch_mask.append([1] * (l) + [0] * (max_len - l))  # 0代表padding的位置，需要mask
    for item in indices:
        # ['prev_items', 'locale', 'recall', 'next_item', 'seqs_cat_feas', 'seqs_num_feas']
        prev_items = item[0].copy()
        while (len(prev_items) < max_len):
            prev_items.append(id_count)  # embdding的时候id_count+1，把id_count作为padding了
        batch_prev_items.append(prev_items)
        batch_locale.append(item[1])
        batch_candidate_set.append(item[2].copy())
        batch_label.append(item[3])
        if item[3] in item[2]:
            batch_label_index.append(item[2].index(item[3]))
        else:
            batch_label_index.append(len(item[2]))
        batch_seq_cat.append(item[4])
        batch_seq_num.append(item[5])
    return [torch.LongTensor(batch_prev_items).to(device), torch.LongTensor(batch_locale).to(device), 
            torch.LongTensor(batch_candidate_set).to(device),
            torch.LongTensor(batch_len).to(device), torch.LongTensor(batch_mask).to(device), 
            torch.LongTensor(batch_label).to(device), torch.LongTensor(batch_label_index).to(device),
            torch.LongTensor(batch_seq_cat).to(device), torch.FloatTensor(batch_seq_num).to(device)]

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, num_workers=num_workers)
logger.info('train_loader: {}'.format(len(train_loader)))
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, num_workers=num_workers)
logger.info('test_loader: {}'.format(len(test_loader)))

output_dir = 'ckpt/{}'.format(exp_id)
ensure_dir(output_dir)

if load_init:
    assert len(load_exp_id) == len(load_epoch)


logger.info('Testing...')
test_scores_all = []
test_res_all = []
for index, exp_id_index in enumerate(load_exp_id):
    logger.info('Index {}'.format(index))
    epoch_index = load_epoch[index]
    fold_index = load_Fold[index]
    model_name_index = 'MatchModelV2withATTMatchFold{}'.format(fold_index)

    load_dir = 'ckpt/{}'.format(exp_id_index)
    load_path = '{}/{}_{}_{}.pt'.format(load_dir, exp_id_index, model_name_index, epoch_index)
    logger.info('Load Init model from {}'.format(load_path))
    model.load_state_dict(torch.load(load_path, map_location='cpu'))
    model.to(device)

    # 开始评估
    test_scores = []
    test_res = []
    model.eval()
    for batch_prev_items, batch_locale, batch_candidate_set, batch_len, batch_mask, \
            _, batch_label_index, batch_seq_cat, batch_seq_num in tqdm(test_loader, desc='test model {}'.format(exp_id), total=len(test_loader)):
        score, _ = model.predict(batch_prev_items=batch_prev_items, batch_locale=batch_locale, 
                                        batch_candidate_set=batch_candidate_set, batch_len=batch_len, 
                                        batch_label=batch_label_index, batch_mask=batch_mask,
                                        batch_seq_cat=batch_seq_cat, batch_seq_num=batch_seq_num)  # (batch_size, 10)
        test_scores.append(score.detach().cpu().numpy())
        sorted_indices = torch.argsort(score, dim=1, descending=True)
        sorted_candidate_set = batch_candidate_set.gather(dim=1, index=sorted_indices)  # (B, 10)
        test_res.append(sorted_candidate_set.detach().cpu().numpy())
    test_scores = np.concatenate(test_scores, axis=0)  # (N, 10)
    test_res = np.concatenate(test_res, axis=0)
    test_scores_all.append(test_scores)
    test_res_all.append(test_res_all)

2023-05-17 11:04:08,115 - INFO - Log directory: ./log
2023-05-17 11:04:08,116 - INFO - Exp_id 85406
2023-05-17 11:04:08,116 - INFO - {'__name__': '__main__', '__doc__': '\n相比于trainDeeep.py，加入一些手动聚合的序列特征，例如历史序列的平均价格，历史序列的不同类别数之类的\nSeqFeatureEmbedding现在使用2层全连接，可以换多层！\n# TODO: product-id作为特征之一\n# TODO: emb_dim变大 不同特征用不同emb_dim等\n# TODO: DIN等序列模型\n', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', 'import numpy as np \nimport pandas as pd\n# import matplotlib.pyplot as plt\n# import seaborn as sns\n# import os\n# import re\n# import math\n# import plotly.express as px\n# import plotly.graph_objects as go\n# from plotly.subplots import make_subplots\n\n# from lightgbm import LGBMRegressor, LGBMClassifier\n# from xgboost import XGBRegressor, XGBClassifier\n# from catboost import CatBoostRegressor, CatBoostClassifier\n# import lightgbm as lgb\n# import xgboost as xgb\n# import

100%|██████████| 316971/316971 [00:15<00:00, 19811.29it/s] 

2023-05-17 11:06:45,010 - INFO - Eval the prev_items


2023-05-17 11:07:37,806 - INFO - df_test: (316971, 2)
2023-05-17 11:07:38,300 - INFO - data_feature:
2023-05-17 11:07:38,301 - INFO - {'len_encode_brand': 177190, 'len_encode_color': 203261, 'len_encode_size': 218061, 'len_encode_model': 524102, 'len_encode_material': 45569, 'len_encode_author': 30836, 'len_locale': 6, 'dense_bins': 10, 'id_count': 1410675, 'len_features': 13, 'len_emb_features': 3, 'len_candidate_set': 100, 'w2v_vector_size': 128, 'sentence_vector_size': 384, 'len_seqs_cat_feas': 14, 'len_seqs_num_feas': 15, 'seq_emb_factor': 4, 'idNUNIQUE': 133, 'idCOUNT': 475, 'brandNUNIQUE': 39, 'brandCOUNT': 475, 'colorNUNIQUE': 54, 'colorCOUNT': 475, 'sizeNUNIQUE': 116, 'sizeCOUNT': 475, 'modelNUNIQUE': 59, 'modelCOUNT': 475, 'materialNUNIQUE': 26, 'materialCOUNT': 475, 'authorNUNIQUE': 65, 'authorCOUNT': 475}
2023-05-17 11:07:38,303 - INFO - create model
2023-05-17 11:07:55,913 - INFO - MatchModelV2(
  (product_emb): ProductEmbedding(
    (product_fea): Product(
      (locale_em

test model 85406:   0%|          | 0/310 [00:03<?, ?it/s]


RuntimeError: cuDNN error: CUDNN_STATUS_INTERNAL_ERROR